### Notebook: functions to take raw .csvs downloaded from sheet and put them into a readable format. Output files are in `Processed CSVs` folder

In [3]:
# Imports
import pandas as pd
import numpy as np


In [52]:
def make_blocks(data):
    '''
    Function that takes raw df with "Trace name" as in position [0,0] and returns the blocks
    '''
    # Each block has 5 columns: Trace name, ID, On time, Freq, Tags
    num_cols = data.shape[1]
    num_rows = data.shape[0]    
    block_size = 5
    blocks = []
    for start in range(0, num_cols, block_size):  
        end = start + block_size 
        if (end <= num_cols) & (data.iloc[0,start] == "Trace name"):
            block = data.iloc[:, start:end]
            # print(block)
            block.columns = ['Trace name', 'ID', 'On time', 'Freq', 'Tags']
            if start == 0:
                block['Type']=['Inhibitory']*num_rows
            elif start == 5:
                block['Type']=['Excitatory']*num_rows
            elif start == 10:
                block['Type']=['Cell-attached']*num_rows
            elif start == 15:
                block['Type']=['Cell-attached (spiking)']*num_rows
            elif start == 20:
                block['Type']=['Inhibitory (Rs compensation)']*num_rows
            elif start == 25:
                block['Type']=['Excitatory (Rs compensation)']*num_rows   
            blocks.append(block)
    # Combine all blocks into one DataFrame
    df_clean = pd.concat(blocks, ignore_index=True)

    # Drop rows that have NaN in Trace name or ID
    df_clean.dropna(subset=['Trace name', 'ID'], how='all', inplace=True)

    df_clean = df_clean.dropna(how='all')
    return df_clean


def prepare_df(df):
    '''
    Function to convet column data types to feed make_ waveworms function
    '''
    df['Trace name'] = df['Trace name'].astype('str')
    df["On time"] = pd.to_numeric(df["On time"], errors="coerce")
    df["Freq"] = pd.to_numeric(df["Freq"], errors="coerce")
    df["Seconds"] = df['On time']*0.001
    return df


def trace_filter(df):
    pattern = r'^\d{4}_\d{2}_\d{2}_\d{4}$'
    index = df['Trace name'].str.match(pattern, na=False)
    df_filtered = df[index]
    return df_filtered

def create_df(file_path):
    '''
    Complete function that takes csv file path from excel spreadsheets and turns it into a readable dataframe that show event times

    '''

    df = pd.read_csv(file_path,header=None)

    # Read the raw CSV , headers are irregular
    data = df.iloc[1:,1:]
    # Keep column spacer between spikes and inhibitory Rs
    # data.iloc[:, 21] = data.iloc[:, 21].astype('object')  # Allow mixed types
    if data.shape[1]>21:
        data.iloc[0,21]= 21
    # Now drop all other NaN columns
    data = data.dropna(axis=1, how='all')
    data = data.dropna(axis=0, how='all') # drop NaN rows

    # Run all of the functions to process files
    df_clean = make_blocks(data)
    prep_df = prepare_df(df_clean)
    df_filtered = trace_filter(prep_df)
    df_filtered = df_filtered.dropna(how='all')

    return df_filtered

In [5]:
sheet_names_df = pd.read_csv('/Users/haleyoro/Desktop/murray-neuroscience-lab/Processed CSVs/sheet_names.csv', header=None)
sheet_names = sheet_names_df.iloc[:,0].values
sheet_names

array(['2012_04_25_cell1', '2012_04_25_cell3', '2012_04_27_cell1',
       '2012_06_22_cell3', '2012_06_25_cell3', '2012_06_29',
       '2012_08_01_cell1', '2012_08_01_cell3', '2012_08_07',
       '2012_08_31_cell1', '2012_08_31_cell2', '2012_08_31_cell3',
       '2012_08_31_cell4', '2012_10_04_cell2', '2012_12_03_cell1',
       '2012_12_04_cell1', '2012_12_04_cell2', '2012_12_05_cell1',
       '2012_12_05_cell3', '2012_12_05_cell4', '2012_12_06_cell1',
       '2012_12_06_cell2', '2012_12_06_cell3', '2012_12_06_cell4',
       '2012_12_06_cell5', '2012_12_06_cell6', '2013_03_20_cell1',
       '2013_03_21_cell1', '2013_03_21_cell2', '2013_03_21_cell3',
       '2013_03_21_cell4', '2013_03_21_cell5', '2013_03_22_cell1',
       '2013_03_22_cell2', '2013_03_22_cell3', '2013_03_22_cell4',
       '2013_03_22_cell5', '2013_03_22_cell6', '2012_10_04_cell1'],
      dtype=object)

Load names of all .abf files in the shared Google folder

In [54]:
abfs = pd.read_csv('/Users/haleyoro/Desktop/murray-neuroscience-lab/Excel processor/all_abf_files.csv',header=None)
abfs = abfs[0].to_numpy()
abfs


array(['2013_03_22_0045.abf', '2013_03_22_0042.abf',
       '2013_03_22_0039.abf', ..., '2012_08_29_0009.abf',
       '2012_08_29_0016.abf', '2012_08_29_0018.abf'],
      shape=(1050,), dtype=object)

In [53]:
df_filtered = create_df('/Users/haleyoro/Downloads/Exported_CSVs/2012_12_04_cell2.csv')
df_filtered

,Trace name,ID,On time,Freq,Tags,Type,Seconds
219,2012_12_04_0021,1,1570.63,NaN,bout start; shock,Cell-attached,1.57063
220,2012_12_04_0021,2,1596.62,38.4763,NaN,Cell-attached,1.59662
221,2012_12_04_0021,3,1624.47,35.9066,bout end,Cell-attached,1.62447
222,2012_12_04_0021,4,1687.42,15.8856,bout start; shock,Cell-attached,1.68742
223,2012_12_04_0021,5,1720.21,30.4971,NaN,Cell-attached,1.72021
...,...,...,...,...,...,...,...
603,2012_12_04_0024,6,42668.80,27.3973,NaN,Excitatory (Rs compensation),42.66880
604,2012_12_04_0024,7,42705.86,26.9833,NaN,Excitatory (Rs compensation),42.70586
605,2012_12_04_0024,8,42742.40,27.3673,NaN,Excitatory (Rs compensation),42.74240
606,2012_12_04_0024,9,42775.92,29.8329,NaN,Excitatory (Rs compensation),42.77592


In [55]:
df_filtered = create_df('/Users/haleyoro/Downloads/Exported_CSVs/2012_10_04_cell2.csv')
df_filtered

,Trace name,ID,On time,Freq,Tags,Type,Seconds
1,2012_10_04_0029,1,12799.27,NaN,bout start; shock,Inhibitory,12.79927
2,2012_10_04_0029,2,12818.74,51.3611,NaN,Inhibitory,12.81874
3,2012_10_04_0029,3,12835.24,60.6061,NaN,Inhibitory,12.83524
4,2012_10_04_0029,4,12854.46,52.0291,NaN,Inhibitory,12.85446
5,2012_10_04_0029,5,12874.93,48.8520,NaN,Inhibitory,12.87493
...,...,...,...,...,...,...,...
1008,2012_10_04_0024,45,27858.82,37.3692,NaN,Cell-attached (spiking),27.85882
1009,2012_10_04_0024,46,27862.02,312.5000,NaN,Cell-attached (spiking),27.86202
1010,2012_10_04_0024,47,35639.53,0.1286,NaN,Cell-attached (spiking),35.63953
1011,2012_10_04_0024,48,35661.83,44.8430,NaN,Cell-attached (spiking),35.66183


One function to run all the functions together

In [56]:
for name in sheet_names:
    file_path = "/Users/haleyoro/Downloads/Exported_CSVs/"+name + '.csv'
    df_filtered = create_df(file_path)
    df_filtered.to_csv(name+".csv", index=False)

/var/folders/40/qf3btjqs32l1pmzzyts7w9pmh2xrdg/T/ipykernel_53653/1100420938.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  block['Type']=['Inhibitory']*num_rows
/var/folders/40/qf3btjqs32l1pmzzyts7w9pmh2xrdg/T/ipykernel_53653/1100420938.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  block['Type']=['Excitatory']*num_rows
/var/folders/40/qf3btjqs32l1pmzzyts7w9pmh2xrdg/T/ipykernel_53653/1100420938.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try